In [1]:
# -*- coding: utf-8 -*- 
from lib import geoSearch
import json
import logging

/usr/local/lib/python2.7/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
GS = geoSearch.GeoSearch("elasticsearch-geo",9200)
geoSearch.logger.setLevel(logging.DEBUG)
from shapely.geometry import mapping, shape

In [3]:
#GS.test_kit('test_complete.csv')

In [4]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'elasticsearch-geo', 'port': 9200}])
from elasticsearch.client import IndicesClient
ic = IndicesClient(es)
def analyzedText(text):
    results = ic.analyze(index="montevideo", body=
    {
      "text": text, 
        "analyzer":"calle_analyzer"
    })
    return ' '.join([token['token'] for token in results['tokens']])

In [7]:
text = u"beatriztalegon separacion d poderes total hemos descabezado al independentismo lo jodido es q todos los jubilados sin redes q se tragan la basura tv n votaran ciudadanos"
print(analyzedText(text))

beatriztalegon separacion separacion_d d_poderes poderes total descabezado descabezado_al al_independentismo independentismo jodido todos_los los_jubilados jubilados redes tragan tragan_la la_basura basura votaran ciudadanos


In [8]:
matches = GS.complete_search(text,50)
print(GS.strip_name(text, ' '))
for key, match in matches:
    print('✔️ MATCH: {:.3f}{: >20}{: >50}'.format(
        match['score'], 
        match['field'], 
        match['name']
    ))
print('\n')

DEBUG:GeoSearch:sin nombre nuevo capra # los pinos ## 11.318093 
 -->   text_second_street
              normal        LOS => 0.333
-->    text_first_street
              normal        SIN => 0.136

DEBUG:GeoSearch:FIELD:text_first_street
min_idx = 114
max_idx = 114
len_longest = 3
highlights_count = 1
importance = 0.136363636364
DEBUG:GeoSearch:FIELD:text_second_street
min_idx = 100
max_idx = 100
len_longest = 3
highlights_count = 1
importance = 0.333333333333
DEBUG:GeoSearch:len_exclusive_first = 1
len_exclusive_secnd = 1
DEBUG:GeoSearch:TOTAL = 869.903015702
First Score = 13.8903868636
Second Score = 33.954279
Boost Field = 100
Distance = 11
DEBUG:GeoSearch:los cedros # los nogales ## 8.926407 
 -->   text_second_street
              normal        LOS => 0.273
-->    text_first_street
              normal        LOS => 0.300

DEBUG:GeoSearch:FIELD:text_first_street
min_idx = 100
max_idx = 100
len_longest = 3
highlights_count = 1
importance = 0.3
DEBUG:GeoSearch:FIELD:text_second_str

DEBUG:GeoSearch:len_exclusive_first = 0
len_exclusive_secnd = 0
DEBUG:GeoSearch:los libustros # los dorados ## 8.926407 
 -->   text_second_street
              normal        LOS => 0.273
-->    text_first_street
              normal        LOS => 0.231

DEBUG:GeoSearch:FIELD:text_first_street
min_idx = 100
max_idx = 100
len_longest = 3
highlights_count = 1
importance = 0.230769230769
DEBUG:GeoSearch:FIELD:text_second_street
min_idx = 100
max_idx = 100
len_longest = 3
highlights_count = 1
importance = 0.272727272727
DEBUG:GeoSearch:len_exclusive_first = 0
len_exclusive_secnd = 0
DEBUG:GeoSearch:los teros # los dorados ## 8.926407 
 -->   text_second_street
              normal        LOS => 0.273
-->    text_first_street
              normal        LOS => 0.333

DEBUG:GeoSearch:FIELD:text_first_street
min_idx = 100
max_idx = 100
len_longest = 3
highlights_count = 1
importance = 0.333333333333
DEBUG:GeoSearch:FIELD:text_second_street
min_idx = 100
max_idx = 100
len_longest = 3
highlight

DEBUG:GeoSearch:los arces # los aromos ## 8.926407 
 -->   text_second_street
              normal        LOS => 0.300
-->    text_first_street
              normal        LOS => 0.333

DEBUG:GeoSearch:FIELD:text_first_street
min_idx = 100
max_idx = 100
len_longest = 3
highlights_count = 1
importance = 0.333333333333
DEBUG:GeoSearch:FIELD:text_second_street
min_idx = 100
max_idx = 100
len_longest = 3
highlights_count = 1
importance = 0.3
DEBUG:GeoSearch:len_exclusive_first = 0
len_exclusive_secnd = 0
DEBUG:GeoSearch:los pioneros # los tumbos ## 8.926407 
 -->   text_second_street
              normal        LOS => 0.300
-->    text_first_street
              normal        LOS => 0.250

DEBUG:GeoSearch:FIELD:text_first_street
min_idx = 100
max_idx = 100
len_longest = 3
highlights_count = 1
importance = 0.25
DEBUG:GeoSearch:FIELD:text_second_street
min_idx = 100
max_idx = 100
len_longest = 3
highlights_count = 1
importance = 0.3
DEBUG:GeoSearch:len_exclusive_first = 0
len_exclusive_secnd

DEBUG:GeoSearch:FIELD:text_first_street
min_idx = 100
max_idx = 100
len_longest = 3
highlights_count = 1
importance = 0.157894736842
DEBUG:GeoSearch:FIELD:text_second_street
min_idx = 100
max_idx = 100
len_longest = 3
highlights_count = 1
importance = 0.272727272727
DEBUG:GeoSearch:len_exclusive_first = 0
len_exclusive_secnd = 0


beatriztalegon separacion d poderes total hemos descabezado al independentismo lo jodido es q todos los jubilados sin redes q se tragan la basura tv n votaran ciudadanos
✔️ MATCH: 869.903         cruces_vias                  sin-nombre-nuevo-capra#los-pinos




In [ ]:
for res in GS.search_cruces_vias(text, 'cruces_vias',50):
    obj = GS.get_result_object(res)
    print(GS.print_object(obj))

In [ ]:
print(matches)

In [ ]:
\n